In [2]:
import pandas as pd

In [20]:
dim_cliente = pd.read_parquet('C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/3_gold/dim_clientes.parquet')
dim_loja = pd.read_parquet('C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/3_gold/dim_loja.parquet')
dim_produto = pd.read_parquet('C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/3_gold/dim_produtos.parquet')
dim_tempo = pd.read_parquet('C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/3_gold/dim_tempo.parquet')
fato_devolucao = pd.read_parquet('C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/3_gold/fato_devolucao.parquet')
fato_vendas = pd.read_parquet('C:/Users/Kimura/Desktop/Projetos/Analise_Performance_Comercial_Global/3_gold/fato_vendas.parquet')

# Análises Propostas
- 1 - Faturamento por país e continente;
- 2 - Ranking dos produtos mais vendidos;
- 3 - Ranking dos produtos mais devolvidos;
- 4 - Taxa de devolução (%) por produto;
- 5 - Ticket médio por cliente;
- 6 - Top 10 lojas com maior faturamento;
- 7 - Top 10 lojas com maior devolução;
- 8 - Identificação de produtos vendidos com prejuízo.

### 1 - Faturamento por país e continente

In [22]:
fato_vendas.info()
dim_loja.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2630 entries, 0 to 2629
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ordem_de_compra  2630 non-null   object        
 1   data_da_venda    2630 non-null   datetime64[ns]
 2   id_cliente       2630 non-null   int64         
 3   id_loja          2630 non-null   int64         
 4   sku              2630 non-null   object        
 5   qtd_vendida      2630 non-null   int64         
 6   preco_unitario   2630 non-null   float64       
 7   custo_unitario   2630 non-null   float64       
 8   receita          2630 non-null   float64       
 9   lucro            2630 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(2)
memory usage: 205.6+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 293 entries, 0 to 1767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------         

In [30]:
df_fat_local = fato_vendas.merge(dim_loja[['id_loja','pais','continente']], on='id_loja',how='left')
df_fat_local.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2630 entries, 0 to 2629
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ordem_de_compra  2630 non-null   object        
 1   data_da_venda    2630 non-null   datetime64[ns]
 2   id_cliente       2630 non-null   int64         
 3   id_loja          2630 non-null   int64         
 4   sku              2630 non-null   object        
 5   qtd_vendida      2630 non-null   int64         
 6   preco_unitario   2630 non-null   float64       
 7   custo_unitario   2630 non-null   float64       
 8   receita          2630 non-null   float64       
 9   lucro            2630 non-null   float64       
 10  pais             2630 non-null   object        
 11  continente       2630 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(4)
memory usage: 246.7+ KB


In [38]:
fat_continente = (
    df_fat_local.groupby('continente')['receita']
    .sum()
    .reset_index()
    .sort_values(by='receita',ascending=False)
)
fat_continente

,continente,receita
0,América do Norte,1308264.78
3,Ásia,451228.77
1,Europa,404892.16
2,Oceania,19374.70


## 2 - Ranking dos produtos mais vendidos (por quantidade)

In [61]:
df_prod_vendas = fato_vendas.merge(dim_produto[['sku','produto']],on='sku',how='left')

rank_produtos = (
    df_prod_vendas.groupby('produto')
    .agg(
        qtd_vendida=('qtd_vendida','sum'),
        valor_total_vendas=('receita','sum')
    )
    .reset_index()
    .sort_values(by='qtd_vendida',ascending=False)
)

rank_produtos

,produto,qtd_vendida,valor_total_vendas
39,iPhone 13 Pro Max 512GB - Vermelho,179,250421.00
36,iPhone 13 Pro Max 128GB - Preto,169,185731.00
37,iPhone 13 Pro Max 1TB - Vermelho,168,268632.00
35,iPhone 13 Pro 128GB - Vermelho,157,156843.00
38,iPhone 13 Pro Max 256GB - Branco,139,166661.00
7,HP Pavilion 17.3' - Preto,108,86292.00
3,Chromebook Spin 311 - Vermelho,106,24062.00
8,Inspiron 3000 16GB RAM - Vermelho,90,46080.00
11,Microsoft Surface Pro 7 12.3' - Preto,83,35607.00
2,Chromebook S330 14' - Vermelho,80,15088.80


In [63]:
top_10_prod_mais_vendidos = rank_produtos.head(10)
top_10_prod_mais_vendidos

,produto,qtd_vendida,valor_total_vendas
39,iPhone 13 Pro Max 512GB - Vermelho,179,250421.0
36,iPhone 13 Pro Max 128GB - Preto,169,185731.0
37,iPhone 13 Pro Max 1TB - Vermelho,168,268632.0
35,iPhone 13 Pro 128GB - Vermelho,157,156843.0
38,iPhone 13 Pro Max 256GB - Branco,139,166661.0
7,HP Pavilion 17.3' - Preto,108,86292.0
3,Chromebook Spin 311 - Vermelho,106,24062.0
8,Inspiron 3000 16GB RAM - Vermelho,90,46080.0
11,Microsoft Surface Pro 7 12.3' - Preto,83,35607.0
2,Chromebook S330 14' - Vermelho,80,15088.8


In [65]:
top_10_prod_menos_vendidos = rank_produtos.tail(10)
top_10_prod_menos_vendidos

,produto,qtd_vendida,valor_total_vendas
5,HP 15 Laptop - Amarelo,27,14283.00
19,iPhone 12 512GB - Preto,26,24674.00
10,Lenovo IdeaPad 3 Gamer - Amarelo,26,25739.74
6,HP Pavilion 15.6' - Amarelo,26,9334.00
18,iPhone 12 256GB - Preto,26,19474.00
26,iPhone 12 Pro 512GB - Prata,25,28725.00
32,iPhone 13 Mini 128GB - Vermelho,24,16776.00
24,iPhone 12 Pro 1TB - Prata,24,32376.00
40,iPhone SE 128GB - Vermelho,24,11496.00
17,iPhone 12 128GB - Prata,22,14278.00


## 3 - Ranking dos produtos mais devolvidos;

In [85]:
df_prod_devolucao = fato_devolucao.merge(dim_produto[['sku']],on='sku',how='left')

rank_produtos_devolvidos = (
    df_prod_devolucao.groupby('produto')
    .agg(
        qtd_vendida=('qtd_devolvida','sum'),
        valor_total_vendas=('valor_devolvido','sum')
    )
    .reset_index()
    .sort_values(by='qtd_vendida',ascending=False)
)
rank_produtos_devolvidos

,produto,qtd_vendida,valor_total_vendas
71,Mouse sem fio MO251 2.4 Ghz - Preto,155,773.45
94,Teclado sem fio V320 - Preto,95,10450.00
70,Mouse sem fio M185 - Preto,93,1487.07
73,Mouse Óptico USB 1200dpi - Preto,77,307.23
44,M510 Mouse sem fio - Vermelho,70,1816.50
...,...,...,...
77,Notebook Asus ROG Zephyrus G15 - Azul,1,1999.00
100,iPhone 12 256GB - Preto,1,749.00
105,iPhone 12 Pro 128GB - Preto,1,849.00
87,Razer Blade 14 - Preto,1,1999.00


In [89]:
top_10_devolucao = rank_produtos_devolvidos.head(10)
top_10_devolucao

,produto,qtd_vendida,valor_total_vendas
71,Mouse sem fio MO251 2.4 Ghz - Preto,155,773.45
94,Teclado sem fio V320 - Preto,95,10450.00
70,Mouse sem fio M185 - Preto,93,1487.07
73,Mouse Óptico USB 1200dpi - Preto,77,307.23
44,M510 Mouse sem fio - Vermelho,70,1816.50
92,Teclado LG X130 - Branco,67,670.00
68,Mouse sem fio D09 - Azul,66,791.34
65,Mouse M90 1000dpi - Preto,56,391.44
67,Mouse com fio USB M90 - Preto,54,431.46
69,Mouse sem fio M170 - Preto,52,1037.40


## 4 - Taxa de devolução (%) por produto

In [124]:
# Total de vendas por SKU:
vendas_sku = fato_vendas.groupby('sku')['qtd_vendida'].sum().reset_index()

# Total de devoluções por SKU:
devolucoes_sku = fato_devolucao.groupby('sku')['qtd_devolvida'].sum().reset_index()

# DF com essas duas bases:
df_taxa_devolucao = vendas_sku.merge(devolucoes_sku, on='sku',how='left')

# Tratando valores nulos na tabela:
df_taxa_devolucao['qtd_devolvida'] = df_taxa_devolucao['qtd_devolvida'].fillna(0)

# Calculando a taxa:
df_taxa_devolucao['taxa_devolucao_%'] = round(
    (df_taxa_devolucao['qtd_devolvida'] / df_taxa_devolucao['qtd_vendida'])*100,2
)

# Join com o nome do produto:
df_taxa_devolucao = df_taxa_devolucao.merge(dim_produto[['sku', 'produto']], on='sku', how='left')

# Ordenar por maior taxa de devolução
df_taxa_devolucao = df_taxa_devolucao.sort_values(by='taxa_devolucao_%', ascending=False)




In [126]:
df_taxa_devolucao.info()
df_taxa_devolucao.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 42 to 14
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sku               44 non-null     object 
 1   qtd_vendida       44 non-null     int64  
 2   qtd_devolvida     44 non-null     float64
 3   taxa_devolucao_%  44 non-null     float64
 4   produto           44 non-null     object 
dtypes: float64(2), int64(1), object(2)
memory usage: 2.1+ KB


,sku,qtd_vendida,qtd_devolvida,taxa_devolucao_%,produto
42,HL94,26,10.0,38.46,HP Pavilion 15.6' - Amarelo
41,HL93,33,11.0,33.33,Samsung 15.6' Galaxy Book2 Pro - Amarelo
28,HL76,50,15.0,30.00,iPhone 11 128GB - Preto
39,HL91,26,7.0,26.92,Lenovo IdeaPad 3 Gamer - Amarelo
29,HL77,78,21.0,26.92,iPhone 11 256GB - Preto
25,HL73,64,17.0,26.56,iPhone 12 Mini 512GB - Prata
10,HL58,24,6.0,25.00,iPhone SE 128GB - Vermelho
27,HL75,50,12.0,24.00,iPhone 11 Pro 256GB - Prata
26,HL74,64,15.0,23.44,iPhone 11 Pro 128GB - Prata
30,HL78,78,18.0,23.08,iPhone XS 64GB - Preto


## 5 - Ticket médio por cliente

In [134]:
# Iremos agrupar os gastos por cliente:

gasto_cliente = (
    fato_vendas
    .groupby('id_cliente')
    .agg(
        total_gasto = ('receita','sum'),
        total_compras = ('ordem_de_compra','nunique')
        )
        .reset_index()
    )

In [146]:
gasto_cliente['tkt_medio'] = round(
    gasto_cliente['total_gasto'] / gasto_cliente['total_compras'],2
    )

In [148]:
gasto_cliente.info()
gasto_cliente.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2630 entries, 0 to 2629
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_cliente     2630 non-null   int64  
 1   total_gasto    2630 non-null   float64
 2   total_compras  2630 non-null   int64  
 3   tkt_medio      2630 non-null   float64
dtypes: float64(2), int64(2)
memory usage: 82.3 KB


,id_cliente,total_gasto,total_compras,tkt_medio
0,11090,1099.0,1,1099.0
1,11129,1599.0,1,1599.0
2,11171,1399.0,1,1399.0
3,11175,1399.0,1,1399.0
4,11191,1399.0,1,1399.0
5,11216,1399.0,1,1399.0
6,11217,1099.0,1,1099.0
7,11224,1099.0,1,1099.0
8,11227,1599.0,1,1599.0
9,11237,549.0,1,549.0


## 6 - Top 10 lojas com maior faturamento

In [173]:
# Agregando vendas por loja.

faturamento_lojas = (
    fato_vendas
    .groupby('id_loja')
    .agg(valor_total=('receita', 'sum'))
    .reset_index()
)

faturamento_lojas = faturamento_lojas.merge(dim_loja[['id_loja','nome_da_loja','pais','continente']], on='id_loja',how='left')

In [175]:
top_lojas = faturamento_lojas.sort_values(by='valor_total', ascending=False).head(10)
top_lojas


,id_loja,valor_total,nome_da_loja,pais,continente
194,200,149613.15,Loja Catalog,Estados Unidos,América do Norte
288,306,125321.42,Loja Europe Online,Alemanha,Europa
289,307,119091.39,Loja Asia Online,China,Ásia
193,199,115598.13,Loja North America Online,Estados Unidos,América do Norte
292,310,95723.93,Loja Asia Reseller,China,Ásia
290,308,93017.60,Loja North America Reseller,Estados Unidos,América do Norte
291,309,92913.88,Loja Europe Reseller,França,Europa
262,278,13417.99,Loja Urumqi,China,Ásia
275,293,13112.00,Loja Hong Kong No.2,China,Ásia
257,270,11225.99,Loja Osaka No.1,Japão,Ásia


## 7 - Top 10 lojas com maior devolução

In [186]:
# Agrupar devoluções por loja
devolucoes_lojas = (
    fato_devolucao
    .groupby('id_loja')
    .agg(qtd_devolvida=('qtd_devolvida', 'sum'))
    .reset_index()
)

In [192]:
# Trazer nome da loja
devolucoes_lojas = devolucoes_lojas.merge(
    dim_loja[['id_loja', 'nome_da_loja','pais','continente']],
    on='id_loja',
    how='left'
)

In [194]:
top_lojas_devolucao = devolucoes_lojas.sort_values(by='qtd_devolvida', ascending=False).head(10)
top_lojas_devolucao

,id_loja,qtd_devolvida,nome_da_loja,pais,continente
188,200,119,Loja Catalog,Estados Unidos,América do Norte
281,306,107,Loja Europe Online,Alemanha,Europa
282,307,106,Loja Asia Online,China,Ásia
187,199,105,Loja North America Online,Estados Unidos,América do Norte
283,308,91,Loja North America Reseller,Estados Unidos,América do Norte
284,309,66,Loja Europe Reseller,França,Europa
285,310,65,Loja Asia Reseller,China,Ásia
237,254,11,Loja Yerevan,Armênia,Ásia
245,264,11,Loja Ashgabat No.1,Turcomenistão,Ásia
84,92,10,Loja Hillsboro,Estados Unidos,América do Norte


In [178]:
fato_devolucao

,data_devolucao,id_loja,sku,qtd_devolvida,preco_unitario,valor_devolvido,produto,marca,tipo_do_produto
0,2020-01-18,309,HL45,1,1399.00,1399.00,iPhone 13 Pro Max 512GB - Vermelho,Apple,Celular
1,2020-01-18,306,HL43,1,1099.00,1099.00,iPhone 13 Pro Max 128GB - Preto,Apple,Celular
2,2020-01-21,41,HL67,1,1349.00,1349.00,iPhone 12 Pro 1TB - Prata,Apple,Celular
3,2020-01-22,306,HL44,1,1199.00,1199.00,iPhone 13 Pro Max 256GB - Branco,Apple,Celular
4,2020-02-02,309,HL45,1,1399.00,1399.00,iPhone 13 Pro Max 512GB - Vermelho,Apple,Celular
...,...,...,...,...,...,...,...,...,...
1804,2022-06-30,68,HL178,1,12.90,12.90,Casaco Hashtag Eletro Bebê - Branco,Hashtag,Casaco
1805,2022-06-30,112,HL164,1,4.99,4.99,Mouse sem fio MO251 2.4 Ghz - Preto,Multilaser,Mouse
1806,2022-06-30,249,HL164,1,4.99,4.99,Mouse sem fio MO251 2.4 Ghz - Preto,Multilaser,Mouse
1807,2022-06-30,246,HL6,1,25.00,25.00,"Camisa Hashtag Eletro Masculina, G - Branco",Hashtag,Camisa


## 8 - Identificação de produtos vendidos com prejuízo

In [199]:
produtos_prejuizo = fato_vendas[
    fato_vendas['preco_unitario'] < fato_vendas['custo_unitario']
].copy()

In [201]:
produtos_prejuizo = produtos_prejuizo.merge(
    dim_produto[['sku', 'produto']],
    on='sku',
    how='left'
)

In [205]:
produtos_prejuizo = produtos_prejuizo.drop_duplicates(subset=['sku'])
print(produtos_prejuizo[['sku', 'produto', 'preco_unitario', 'custo_unitario']])


Empty DataFrame
Columns: [sku, produto, preco_unitario, custo_unitario]
Index: []
